<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Обзор-и-предобработка-данных" data-toc-modified-id="Обзор-и-предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обзор и предобработка данных</a></span></li><li><span><a href="#Анализ-данных" data-toc-modified-id="Анализ-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Анализ спарсенных с hh.ru данных

**Цель работы**

Перевести спарсенный json в формат csv таблицу через pandas, затем:

* Сгруппировать вакансии по направлениями (DS, DE, Software Engenering, etc.)
* Найти среднюю и медианную зарплату по группам вакансий.
* Найти среднюю и медианную зарплату по каждому региону.
* Найти самую высокооплачиваемую из групп вакансий, исходя из их средних зарплат.
* Определить, существует ли корреляция уровня опыта от зарплаты.
* Определить число должностей в наборе данных.
* Найти топ-10 наиболее часто встречающихся должностей.

## Импорт библиотек

Загрузим библиотеки

In [27]:
import numpy as np
import pandas as pd
import json
import re

## Обзор и предобработка данных

Загрузим содержимое файла

In [28]:
with open('..\Parsing Python\data.json', 'r') as file:
    data = json.load(file)

df_hh = pd.json_normalize(data, record_path='data')

Проверим содержимое

In [29]:
df_hh.head()

,title,work experience,salary,region
0,Go разработчик со знаниями Python,1–3 года,до 1 000 000 KZT на руки,Алматы
1,TechLead / Senior Python Developer,3–6 лет,з/п не указана,Москва
2,Разработчик Solidity/Разработчик Rust/Разработ...,3–6 лет,от 600 000 руб. на руки,Неизвестен
3,Разработчик Python,1–3 года,от 160 000 до 200 000 руб. до вычета налогов,Москва
4,Программист Python/Data Scientist (junior / mi...,не требуется,от 60 000 до 100 000 руб. на руки,Неизвестен


Добавим столбец с валютой.

In [30]:
df_hh.loc[df_hh['salary'].str.contains('KZT'), 'currency'] = 'KZT'

df_hh.loc[df_hh['salary'].str.contains('руб'), 'currency'] = 'RUB'

df_hh.loc[df_hh['salary'].str.contains('USD' or 'usd'), 'currency'] = 'USD'

Удалим строки, где валюта не известна.

In [31]:
df_hh = df_hh.dropna(axis=0).reset_index(drop=True)

In [32]:
df_hh.head()

,title,work experience,salary,region,currency
0,Go разработчик со знаниями Python,1–3 года,до 1 000 000 KZT на руки,Алматы,KZT
1,Разработчик Solidity/Разработчик Rust/Разработ...,3–6 лет,от 600 000 руб. на руки,Неизвестен,RUB
2,Разработчик Python,1–3 года,от 160 000 до 200 000 руб. до вычета налогов,Москва,RUB
3,Программист Python/Data Scientist (junior / mi...,не требуется,от 60 000 до 100 000 руб. на руки,Неизвестен,RUB
4,Junior/Middle Python Developer (Backend) - раз...,1–3 года,от 60 000 до 100 000 руб. на руки,Неизвестен,RUB


Извлечем max и min зарплаты

In [33]:
def new_salary(salary):           
    salary_all = re.findall(r'\s\d+\s\d+\s?\d{0,}', salary)
    if len(salary_all) == 1:
        salary_min = np.float32(''.join(salary_all[0].split()))
        salary_max = salary_min
    else:
        salary_min = np.float32(''.join(salary_all[0].split()))
        salary_max = np.float32(''.join(salary_all[1].split()))
        
    return salary_min, salary_max
    
df_hh['salary_min'], df_hh['salary_max'] = zip(*df_hh['salary'].apply(new_salary))

In [34]:
df_hh.head(10)

,title,work experience,salary,region,currency,salary_min,salary_max
0,Go разработчик со знаниями Python,1–3 года,до 1 000 000 KZT на руки,Алматы,KZT,1000000.0,1000000.0
1,Разработчик Solidity/Разработчик Rust/Разработ...,3–6 лет,от 600 000 руб. на руки,Неизвестен,RUB,600000.0,600000.0
2,Разработчик Python,1–3 года,от 160 000 до 200 000 руб. до вычета налогов,Москва,RUB,160000.0,200000.0
3,Программист Python/Data Scientist (junior / mi...,не требуется,от 60 000 до 100 000 руб. на руки,Неизвестен,RUB,60000.0,100000.0
4,Junior/Middle Python Developer (Backend) - раз...,1–3 года,от 60 000 до 100 000 руб. на руки,Неизвестен,RUB,60000.0,100000.0
5,Low-code developer (Python),1–3 года,от 60 000 руб. на руки,Москва,RUB,60000.0,60000.0
6,Junior Python разработчик (Django),не требуется,от 60 000 руб. до вычета налогов,Краснодар,RUB,60000.0,60000.0
7,"Программист Python, удаленно",1–3 года,от 170 000 руб. на руки,Москва,RUB,170000.0,170000.0
8,Разработчик C# / Python,3–6 лет,от 300 000 до 500 000 руб. на руки,Москва,RUB,300000.0,500000.0
9,Junior back-end разработчик (JavaScript/Python...,не требуется,от 55 000 до 63 000 руб. на руки,Неизвестен,RUB,55000.0,63000.0


Добавим столбец со средней зп в рублях.

In [35]:
KZT_TO_RUB = 7.65
USD_TO_RUB = 60.5

def salary_mean_rub(row):
    currency = row['currency']
    smr = row['salary_mean_rub']
    if currency == 'KZT':
        smr = smr / KZT_TO_RUB
    if currency == 'USD':
        smr = smr * USD_TO_RUB        
    return smr 

df_hh['salary_mean_rub'] = (df_hh['salary_min'] + df_hh['salary_max']) / 2
df_hh['salary_mean_rub'] = round(df_hh.apply(salary_mean_rub, axis=1), 0)

In [37]:
df_hh.head()

,title,work experience,salary,region,currency,salary_min,salary_max,salary_mean_rub
0,Go разработчик со знаниями Python,1–3 года,до 1 000 000 KZT на руки,Алматы,KZT,1000000.0,1000000.0,130719.0
1,Разработчик Solidity/Разработчик Rust/Разработ...,3–6 лет,от 600 000 руб. на руки,Неизвестен,RUB,600000.0,600000.0,600000.0
2,Разработчик Python,1–3 года,от 160 000 до 200 000 руб. до вычета налогов,Москва,RUB,160000.0,200000.0,180000.0
3,Программист Python/Data Scientist (junior / mi...,не требуется,от 60 000 до 100 000 руб. на руки,Неизвестен,RUB,60000.0,100000.0,80000.0
4,Junior/Middle Python Developer (Backend) - раз...,1–3 года,от 60 000 до 100 000 руб. на руки,Неизвестен,RUB,60000.0,100000.0,80000.0


Добавим столбец с направлением вакансии

In [38]:
df_hh['title'] = df_hh['title'].str.lower()

def to_type(title):
    de = ['de', 'data engineer', 'дата инженер', 'инженер данных']
    ds = ['ds', 'data scientist', 'дата саенсист', 'data science']
    dev = ['developer', 'разработчик', 'программист']
    soft = ['software', 'se', 'software']
    type_dict = {'DE': de,
                 'DS': ds,
                 'DEV': dev,
                 'SOFT': soft
                 }

    for name, type_list in type_dict.items():
        for word in type_list:
            if word in title:
                return name  

df_hh['type'] = df_hh['title'].apply(to_type)

In [39]:
df_hh.head()

,title,work experience,salary,region,currency,salary_min,salary_max,salary_mean_rub,type
0,go разработчик со знаниями python,1–3 года,до 1 000 000 KZT на руки,Алматы,KZT,1000000.0,1000000.0,130719.0,DEV
1,разработчик solidity/разработчик rust/разработ...,3–6 лет,от 600 000 руб. на руки,Неизвестен,RUB,600000.0,600000.0,600000.0,DEV
2,разработчик python,1–3 года,от 160 000 до 200 000 руб. до вычета налогов,Москва,RUB,160000.0,200000.0,180000.0,DEV
3,программист python/data scientist (junior / mi...,не требуется,от 60 000 до 100 000 руб. на руки,Неизвестен,RUB,60000.0,100000.0,80000.0,DS
4,junior/middle python developer (backend) - раз...,1–3 года,от 60 000 до 100 000 руб. на руки,Неизвестен,RUB,60000.0,100000.0,80000.0,DE


Удалим лишние столбцы и пустые строки.

In [40]:
df_hh_fin = df_hh.drop(['salary_min', 'salary_max', 'salary', 'currency', 'title'], axis=1)
df_hh_fin = df_hh_fin.dropna(axis=0).reset_index(drop=True)

In [41]:
df_hh_fin.head()

,work experience,region,salary_mean_rub,type
0,1–3 года,Алматы,130719.0,DEV
1,3–6 лет,Неизвестен,600000.0,DEV
2,1–3 года,Москва,180000.0,DEV
3,не требуется,Неизвестен,80000.0,DS
4,1–3 года,Неизвестен,80000.0,DE


Подготовка данных к анализу завершена.

## Анализ данных

Сгруппируем ЗП по типам должностей.

In [42]:
print('Средняя и медиальная ЗП от типа дожности')

pd.pivot_table(df_hh_fin, index='type', aggfunc=['mean', 'median']).sort_values(('mean', 'salary_mean_rub'), ascending=False)

Средняя и медиальная ЗП от типа дожности


,mean,median
,salary_mean_rub,salary_mean_rub
type,,
DS,373888.888889,300000.0
SOFT,274861.111111,181500.0
DE,190840.738636,181500.0
DEV,155544.623457,138062.5


In [43]:
print('Средняя и медиальная ЗП в зависимости от региона')
pd.pivot_table(df_hh_fin, index='region', aggfunc=['mean', 'median']).sort_values(('mean', 'salary_mean_rub'), ascending=False)

Средняя и медиальная ЗП в зависимости от региона


,mean,median
,salary_mean_rub,salary_mean_rub
region,,
Нью-Йорк,650000.000000,650000.0
Химки,300000.000000,300000.0
Воронеж,230000.000000,230000.0
Таганрог,211750.000000,211750.0
Москва,201938.775510,200000.0
Неизвестен,198119.897059,178250.0
Зеленоград,156666.666667,120000.0
Минск,151250.000000,151250.0


In [44]:
print('Средняя и медиальная ЗП в зависимости от опыта')
pd.pivot_table(df_hh_fin, index='work experience', aggfunc=['mean', 'median']).sort_values(('mean', 'salary_mean_rub'), ascending=False)

Средняя и медиальная ЗП в зависимости от опыта


,mean,median
,salary_mean_rub,salary_mean_rub
work experience,,
более 6 лет,284562.500000,271000.0
3–6 лет,226911.991228,200000.0
1–3 года,134495.877049,110000.0
не требуется,62593.750000,60000.0


Найдем общее число уникальных должностей в наборе данных.

In [62]:
uniq_vac = len(df_hh['title'].unique())
print(f"В датасете {uniq_vac} уникальных названий вакансий.")
print(f"Что составляет {uniq_vac / df_hh.shape[0]:.2%} от всего числа вакансий.")

В датасете 212 уникальных названий вакансий.
Что составляет 70.67% от всего числа вакансий.


Проще говоря, названия вакансий не универсализированы.

Найдем 10 наиболее часто встречающихся должностей.

In [58]:
df_hh['title'].value_counts().head(10)

программист python                                         10
python разработчик                                          8
python developer                                            6
инженер-программист                                         6
python developer (middle/senior)                            5
разработчик python                                          5
senior python developer                                     4
аналитик данных | продуктовый аналитик                      3
senior risk data scientist for a global fintech startup     3
ведущий python разработчик                                  3
Name: title, dtype: int64

## Вывод

По результату анализа мы можем сделать следующие выводы:

1. Самое высокооплачиваемое направление - DS. 
2. Самая высокая ЗП в Нью Йорке.
3. Опытным сотрудникам готовы платить в разы больше, чем неопытным.
4. Более 70% названий вакансий не повторяются.
5. Самое популярное название вакансии - "программист python".